Graphing the data from Template_run_Auto.py

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
os.getcwd()
test_title = 'individual' # title of the test, eg. 1,2, A, B, Initial.
field = 'cdfs'
agn_sed = [1]
id_key = 'normal' # normal, high or low

#opening ZFOURGE data
id_key_dict = {'normal': 'inputs/alternate_catalogues/cdfs.range.(0, -1).cat',
      'fraction_bin_10': f'inputs/alternate_catalogues/{field}.fraction.bin10.0.cat',
   'luminosity_bin_10' : f'inputs/alternate_catalogues/{field}.luminosity.bin10.0.cat'}

main_cat = pd.read_csv(id_key_dict[id_key]) # get the catalogue for the id_key
main_cat.to_csv('inputs/eazy_test.cat', index=False) # create a new catalogue, allows for change to be made in this cell

#opening test data
individual_dir = f'outputs/cdfs/{test_title}/'
directory = f'induvidual_data_{field}_{id_key}_0.005_{agn_sed}_True.csv'
df = pd.read_csv(individual_dir+directory)

#Setting up the main catalogue
main = pd.read_csv('inputs/eazy_test.cat', sep=" ", comment="#", header=None, skipinitialspace=True) # opening cut cat, and adjusting it
headers = pd.read_csv('inputs/eazy_test.cat', sep=" ", header=None, nrows=1).iloc[0]
headers = headers[1:]
main.columns = headers
agn_per_dir = f'inputs/{field}_agn_frac.txt' # file with AGN fractions for each object, prepared in catalogue_prepare.ipynb
all_bayes = pd.read_csv(agn_per_dir, sep="\s+", comment='#')
main = pd.merge(main, all_bayes, on='id', how='left') # AGN fraction for each object